In [128]:
import numpy as np
from tkinter import *
import tkinter.scrolledtext as tkst
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
from keras import optimizers
from keras.layers import Dense
from keras.models import Sequential
from keras.datasets import mnist

In [129]:
digit_fig = Figure(figsize=(4,4), dpi=100)
digit_ax = digit_fig.add_subplot(111)
digit_ax.set_title('Selected Digit Image')

major_ticks = np.arange(0, 29, 5)
minor_ticks = np.arange(0, 29, 1)

digit_ax.set_xticks(major_ticks)
digit_ax.set_xticks(minor_ticks, minor=True)
digit_ax.set_yticks(major_ticks)
digit_ax.set_yticks(minor_ticks, minor=True)
digit_ax.grid(which='both')
digit_ax.grid(which='minor', alpha=0.2)
digit_ax.grid(which='major', alpha=0.5)

In [130]:
loss_fig = Figure(figsize=(4,4), dpi=100)
loss_ax = loss_fig.add_subplot(111)
loss_ax.set_xlabel('Epochs')
loss_ax.set_ylabel('Loss')
loss_ax.set_title('History of loss')

Text(0.5, 1.0, 'History of loss')

In [131]:
acc_fig = Figure(figsize=(4,4), dpi=100)
acc_ax = acc_fig.add_subplot(111)
acc_ax.set_xlabel('Epochs')
acc_ax.set_ylabel('Accuracy')
acc_ax.set_title('History of accuracy')

Text(0.5, 1.0, 'History of accuracy')

In [132]:
number_fig = Figure(figsize=(4,4), dpi=100)
number_ax = number_fig.add_subplot(111)
number_ax.set_title('Probability Chart')

Text(0.5, 1.0, 'Probability Chart')

In [133]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# print(x_train.shape)
# print(x_test.shape)

In [134]:
# x_train[0]

In [135]:
def viewImage() :
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_test_original = x_test
    
    t_a = int(t_aSpbox.get()) - 1
    
    selected_image = t_a
    digit_ax.imshow(x_test_original[selected_image])
    digit_fig.canvas.draw()

In [136]:
def learning() :
    import numpy as np
    from keras import optimizers
    from keras.layers import Dense
    from keras.models import Sequential
    
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_test_original = x_test
    
    t_a = int(t_aSpbox.get()) - 1
    t_t = int(t_tSpbox.get())
    selected_image = t_a
    
    model = Sequential()
    
    model.add(Dense(512, activation='relu', input_shape=(28,28)))
    
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    x_train = x_train.reshape((60000, 28 * 28))
    x_train = x_train.astype('float32') / 255
    x_test = x_test.reshape((10000, 28 * 28))
    x_test = x_test.astype('float32') / 255
    
    from keras.utils import to_categorical
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    
    modelCheckpoint = ModelCheckpoint('best_digits_model.hdf5', save_best_only=True)
    monitor_val_acc = EarlyStopping(monitor='val_accuracy', patience=5)
    
    history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=t_t, batch_size=128, callbacks=[monitor_val_acc, modelCheckpoint])
    
    result = model.predict(np.array([x_test[selected_image]]))
    result_number = np.argmax(result)
    
    loss_ax.plot(history.history['loss'], 'ro', label='train loss')
    loss_fig.canvas.draw()
    
    acc_ax.plot(history.history['accuracy'], 'bo', label='train acc')
    acc_fig.canvas.draw()
    
    digits=('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')    
    x_pos = np.arange(len(digits))
    performance = [ val for val in result[0] ]
    
    result_probability = performance[result_number]
    
    number_ax.bar(x_pos, performance, align='center', alpha=0.5)
    
    number_ax.set_title('Number is %2i (probability %7.4f)' % (result_number, result_probability * 100))
    
    number_fig.canvas.draw()
    
    print(performance) 

In [ ]:
main = Tk()
main.title('MNIST Digits, Neural Network')
main.geometry()

t_aVal = IntVar(value=1)
t_aSpbox = Spinbox(main, textvariable=t_aVal, from_=0, to=10000, increment=1)
t_aSpbox.grid(row=0, column=1)
t_aLabel=Label(main, text='Number of Digits : ')
t_aLabel.grid(row=0, column=0)

btn1 = Button(main, text='viewImage', height=1, command=lambda:viewImage())
btn1.grid(row=0, column=2, sticky=(W, E))

t_tVal = IntVar(value=100)
t_tSpbox = Spinbox(main, textvariable=t_tVal, from_=0, to=100000, increment=100, )
t_tSpbox.config(state='readonly')
t_tSpbox.grid(row=0, column=4)
t_tLabel=Label(main, text='Number of trains : ')
t_tLabel.grid(row=0, column=3)

digit_canvas = FigureCanvasTkAgg(digit_fig, main)
digit_canvas.get_tk_widget().grid(row=1, column=0, columnspan=3)

btn2 = Button(main, text='Deep Learning', height=1, command=lambda:learning())
btn2.grid(row=0, column=5, sticky=(W, E))

loss_canvas = FigureCanvasTkAgg(loss_fig, main)
loss_canvas.get_tk_widget().grid(row=1, column=3, columnspan=3)

acc_canvas = FigureCanvasTkAgg(acc_fig, main)
acc_canvas.get_tk_widget().grid(row=2, column=0, columnspan=3)

number_canvas = FigureCanvasTkAgg(number_fig, main)
number_canvas.get_tk_widget().grid(row=2, column=3, columnspan=3)

main.mainloop()

Epoch 1/100


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\JAVA01\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-137-01bb0c7511af>", line 24, in <lambda>
    btn2 = Button(main, text='Deep Learning', height=1, command=lambda:learning())
  File "<ipython-input-136-6161e5b6d2a8>", line 35, in learning
    history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=t_t, batch_size=128, callbacks=[monitor_val_acc, modelCheckpoint])
  File "C:\Users\JAVA01\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\JAVA01\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\JAVA01\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 871, in _call
    self._initialize(args, kwds, ad